In [1]:
# Import the required libraries and dependencies
import pandas as pd
import datetime as dt
from pathlib import Path
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [23]:
spy_df = pd.read_csv(Path("./Resources/spy.csv"))
spy_df= spy_df.drop(columns=['Open','High','Low','Day','Weekday','Week','Month','Year','Volume'])
spy_df






,Date,Close
0,1/2/14,154.989548
1,1/3/14,154.964035
2,1/6/14,154.515045
3,1/7/14,155.464020
4,1/8/14,155.497864
...,...,...
2300,2/22/23,398.540008
2301,2/23/23,400.660004
2302,2/24/23,396.380005
2303,2/27/23,397.730011


In [24]:
spy_df
# spy_df = spy_df.drop('Close')
# spy_df=spy_df.groupby(['Date','Volume'],as_index=False)
# spy_df =spy_df.rename(columns={'Date':'ds','Close':'y'})

,Date,Close
0,1/2/14,154.989548
1,1/3/14,154.964035
2,1/6/14,154.515045
3,1/7/14,155.464020
4,1/8/14,155.497864
...,...,...
2300,2/22/23,398.540008
2301,2/23/23,400.660004
2302,2/24/23,396.380005
2303,2/27/23,397.730011


In [25]:
print(min(spy_df['Date']))
print(max(spy_df['Date']))

1/10/14
9/9/22


In [26]:
spy_df = pd.read_csv(Path("./Resources/spy.csv"),index_col="Date", parse_dates=True, infer_datetime_format=True).dropna()
spy_df= spy_df['Close']
spy_df = spy_df.reset_index()
spy_df.columns= ["ds","y"]
spy_df



,ds,y
0,2014-01-02,154.989548
1,2014-01-03,154.964035
2,2014-01-06,154.515045
3,2014-01-07,155.464020
4,2014-01-08,155.497864
...,...,...
2300,2023-02-22,398.540008
2301,2023-02-23,400.660004
2302,2023-02-24,396.380005
2303,2023-02-27,397.730011


In [28]:

train = spy_df[spy_df['ds'] < '1/3/23']
test = spy_df[spy_df['ds']>='1/3/23']
train_sub = train[train['y'] == 'Close']
test_sub= test[test['y'] == 'Close']
m = Prophet()
m.fit(train_sub)
future = m.make_future_dataframe(periods=365, freq="D")
forecast = m.predict(future)


ValueError: Dataframe has less than 2 non-NaN rows.